In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
movie = '/content/drive/MyDrive/movies.csv'

In [ ]:
rating = '/content/drive/MyDrive/ratings.csv'

In [ ]:
movies = pd.read_csv(movie)

In [ ]:
ratings = pd.read_csv(rating)

In [ ]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
10324,146684,Cosmic Scrat-tastrophe (2015),Animation|Children|Comedy
10325,146878,Le Grand Restaurant (1966),Comedy
10326,148238,A Very Murray Christmas (2015),Comedy
10327,148626,The Big Short (2015),Drama


In [ ]:
ratings

,userId,movieId,rating,timestamp
0,1,16,4.0,1217897793
1,1,24,1.5,1217895807
2,1,32,4.0,1217896246
3,1,47,4.0,1217896556
4,1,50,4.0,1217896523
...,...,...,...,...
105334,668,142488,4.0,1451535844
105335,668,142507,3.5,1451535889
105336,668,143385,4.0,1446388585
105337,668,144976,2.5,1448656898


In [ ]:

def genre_popularity_recommender(g, t, N):
    movies = pd.read_csv(movie)
    ratings = pd.read_csv(rating)

    genre_movies = movies[movies['genres'].str.contains(g)]
    genre_data = pd.merge(genre_movies, ratings, on='movieId')

    movie_ratings = genre_data.groupby('movieId')['rating'].mean()

    movie_review = genre_data.groupby('movieId')['rating'].count()
    movie_stats = pd.DataFrame({'movie_title': genre_movies['title'],'mean_rating': movie_ratings,'review_count': movie_review})
    movie_stats = movie_stats[movie_stats['review_count'] >= t]
    movie_stats = movie_stats.sort_values(by='mean_rating', ascending=False)
    top_movies = movie_stats.head(N)
    top_movies.insert(0, 'S.No', range(1, len(top_movies) + 1))
    return top_movies
  

if __name__=="__main__":
  genre=str(input('Enter genre : '))
  rating_threshold=int(input('Enter rating_threshold : '))
  num_recommendations=int(input('Enter num_recommendations : '))
  print(genre_popularity_recommender(genre, rating_threshold, num_recommendations))

Enter genre : Action
Enter rating_threshold : 12
Enter num_recommendations : 5
      S.No movie_title  mean_rating  review_count
1927     1         NaN     4.500000          13.0
3000     2         NaN     4.384615          52.0
3265     3         NaN     4.307692          13.0
908      4         NaN     4.273973          73.0
5782     5         NaN     4.272727          22.0


In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
def content_based_recommender(movie_title, N):
    movies_data = pd.read_csv(movie)
    features = ['genres']
    for feature in features:
        movies_data[feature] = movies_data[feature].fillna('')
    def combine_features(row):
        return row['genres']
    movies_data["combined_features"] = movies_data.apply(combine_features, axis=1)

    cv = CountVectorizer()
    count_matrix = cv.fit_transform(movies_data["combined_features"])

    cosine_sim = cosine_similarity(count_matrix)

    title_index = movies_data[movies_data["title"] == movie_title]
    if title_index.empty:
        return "Movie title not found in the dataset"
    else:
      title_index = title_index.index[0]

    similarity_scores = list(enumerate(cosine_sim[title_index]))

    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    similarity_scores = similarity_scores[1:N+1]

    movie_indices = [i[0] for i in similarity_scores]

    return movies_data["title"].iloc[movie_indices]

if __name__=="__main__":
  movie_title=str(input('Enter the name of the movie:-'))
  num_recommendations=int(input('Enter the number of recommendations'))
  print(content_based_recommender(movie_title, num_recommendations))


Enter the name of the movie:-Jumanji (1995)
Enter the number of recommendations5
55             Indian in the Cupboard, The (1995)
112             NeverEnding Story III, The (1994)
817               Escape to Witch Mountain (1975)
1604    Darby O'Gill and the Little People (1959)
1652                          Return to Oz (1985)
Name: title, dtype: object


In [ ]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp38-cp38-linux_x86_64.whl size=3366475 sha256=4068ee903b169280a197259f3dacfade1e9760254e0be99544cde5e02255287a
  Stored in directory: /root/.cache/pip/wheels/af/db/86/2c18183a80ba05da35bf0fb7417aac5cddbd93bcb1b92fd3ea
Successfully built scikit-surprise


In [ ]:
import pandas as pd
from surprise import Reader, Dataset, KNNWithMeans

def collaborative_based_recommender(user_id, N, k):
  
    ratings_data = pd.read_csv(rating)
    reader = Reader(rating_scale=(1, 5))
    data = Dataset.load_from_df(ratings_data[['userId', 'movieId', 'rating']], reader)

    trainset = data.build_full_trainset()

    algo = KNNWithMeans(k=k, sim_options={'name': 'pearson_baseline', 'user_based': True})
    algo.fit(trainset)
    predictions = algo.get_neighbors(user_id, k=N)
    movies_data = pd.read_csv(movie)
    recommended_movies = movies_data[movies_data["movieId"].isin(predictions)]
    return recommended_movies.sort_values(by="movieId")[["movieId","title"]]

if __name__=="__main__":
  user_id = int(input('Enter the user-ID'))
  num_recommendations=int(input('Enter the number of recommendations'))
  threshold=int(input('Enter the threshold'))
  print(collaborative_based_recommender(user_id, num_recommendations, threshold))


Enter the user-ID1
Enter the number of recommendations5
Enter the threshold12
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
     movieId                                              title
408      460                       Getting Even with Dad (1994)
514      577                                       Andre (1994)
563      647                          Courage Under Fire (1996)
570      660                                      August (1996)
576      667  Bloodsport 2 (a.k.a. Bloodsport II: The Next K...
